In [1]:
from IPython.core.interactiveshell import InteractiveShell # type: ignore
InteractiveShell.ast_node_interactivity='all'
from ML import *
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
print("GPU is","avaliable" if tf.config.experimental.list_physical_devices("GPU") else "not available")

2024-06-21 14:52:25.142267: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-06-21 14:52:25.142283: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


GPU is not available


2024-06-21 14:52:26.266251: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-06-21 14:52:26.266291: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: kamal-Inspiron-3593
2024-06-21 14:52:26.266296: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: kamal-Inspiron-3593
2024-06-21 14:52:26.266417: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 535.171.4
2024-06-21 14:52:26.266441: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 535.171.4
2024-06-21 14:52:26.266445: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:310] kernel version seems to match DSO: 535.171.4


In [2]:
'''

We first import the necessary libraries and LOAD all the data for training and testing. Such data corresponds to the original samples from DeltaPhish (either benign or malicious) as well as the 100 phishing samples (not used in the training data) that we use as basis to create our adversarial examples.

We then develop a given leaner (i.e., a phishing detector) by using the train data and a Logistic Regression algorithm, and test it on the test data, computing the FPR and TPR shown in Table 3; we also compute the TPR on the subset of 100 samples used for our adversarial evaluations, which correspond to the results in Figure 4b (for the LR algorithm).

We then proceed to assess the robustness of the detector against ALL the 12 attacks considered in our paper, all of which use the 100 samples as basis. For WA and WA+, we already provide the preprocessed version. For PA, we use the samples created during the preprocessing phase. For MA, we create the samples at runtime.
'''

'\n\nWe first import the necessary libraries and LOAD all the data for training and testing. Such data corresponds to the original samples from DeltaPhish (either benign or malicious) as well as the 100 phishing samples (not used in the training data) that we use as basis to create our adversarial examples.\n\nWe then develop a given leaner (i.e., a phishing detector) by using the train data and a Logistic Regression algorithm, and test it on the test data, computing the FPR and TPR shown in Table 3; we also compute the TPR on the subset of 100 samples used for our adversarial evaluations, which correspond to the results in Figure 4b (for the LR algorithm).\n\nWe then proceed to assess the robustness of the detector against ALL the 12 attacks considered in our paper, all of which use the 100 samples as basis. For WA and WA+, we already provide the preprocessed version. For PA, we use the samples created during the preprocessing phase. For MA, we create the samples at runtime.\n'

In [3]:
# The code starts by loading a JSON file containing the dataset using a custom data_load function. 
# This dataset (data_delta) includes features relevant to phishing detection.
json_file='../data_folder/deltaphish/preprocessed/normal/deltaphish_total_feature_modified.json'
data_delta=data_load(json_file)


# remove evasion dataset, then build new PWDs using remain data
# It loads a preprocessed dataset containing 100 samples of phishing data from a pickle file using joblib.
phish_sub_test_x=joblib.load('../data_folder/deltaphish/preprocessed/phish_sub_test_x_100.pkl')
#print('phish_test_x index',phish_sub_test_x.index)#100
#print('phish_sub_test_x shape',phish_sub_test_x.shape)


# The code creates a copy of the original dataset (data_delta_change) to manipulate it without altering the original data.
data_delta_change=data_delta.copy()

#get the 100 phish samples and add them to the test dataset
phish_sub_test_later=data_delta_change.loc[phish_sub_test_x.index].copy()
data_delta_change=data_delta_change.drop(phish_sub_test_x.index)# del those 100 samples


# The code splits the remaining data (data_delta_change) into training and test sets using train_test_split. 
# It sets a fixed training size of 3200 samples and a test size of 700 samples.
# Features (train_x0) and labels (train_y0) for the training set are separated, with features consisting of the first 58 columns and labels being the 59th column.
train_random,test_random=train_test_split(data_delta_change,train_size=3200,test_size=700,shuffle=True,random_state=11)
train_x0=train_random.iloc[:,0:58]
train_y0=train_random.iloc[:,58]




# The code combines the initially separated test data (test_random) with the phishing sub test data (phish_sub_test_later).
# Features (test_x0) and labels (test_y0) for the combined test set are separated similarly to the training set.
#test_later=pd.concat(test_random,phish_sub_test_later)
frames =[test_random,phish_sub_test_later]
test_later=pd.concat(frames)
test_x0=test_later.iloc[:,0:58] 
test_y0=test_later.iloc[:,58] 


# The code extracts HTML-related features (columns 27 to 49) for both training and test sets.
# It also extracts URL-related features (columns 0 to 27 and 49 to 58) for both training and test sets.
# The labels for these subsets are set to the same values as train_y0 and test_y0
#use only html features
train_html_x0=train_random.iloc[:,27:49]
test_html_x0=test_later.iloc[:,27:49]
#only url features
train_url_x0=pd.concat([train_random.iloc[:,0:27],train_random.iloc[:,49:58]],axis=1)
test_url_x0=pd.concat([test_later.iloc[:,0:27],test_later.iloc[:,49:58]],axis=1)
train_y1=train_url_y0=train_html_y0=train_y0
test_y1=test_url_y0=test_html_y0=test_y0


# The code identifies indices of benign and phishing samples in the test data.
benign_index=test_y0[test_y0==0].index.tolist()#
phish_index=test_y0[test_y0==1].index.tolist()#

#generate benign_test_x_y
# It creates separate datasets for benign samples (benign_test_x and benign_test_y) and phishing samples (phish_sub_test_y).
# It extracts HTML and URL features for these benign and phishing test datasets.
benign_test_x=test_x0.loc[benign_index,:]
benign_test_y= test_y0.loc[benign_index]# 
benign_test_url_x=pd.concat([benign_test_x.iloc[:,0:27],benign_test_x.iloc[:,49:58]],axis=1)
benign_test_html_x=benign_test_x.iloc[:,27:49]

phish_sub_test_y=test_y0[phish_sub_test_x.index]#100 phishing samples
phish_sub_test_html_x=phish_sub_test_x.iloc[:,27:49]
phish_sub_test_url_x=pd.concat([phish_sub_test_x.iloc[:,0:27],phish_sub_test_x.iloc[:,49:58]],axis=1)

 

# Baseline Performance- refer to Table 4, bottom row. No Attack and no Evasion space involved

In [4]:
# The code sets a fixed random state for reproducibility. Using a consistent random state ensures that the results can be replicated.
random_state = 42
# Setting the seed for numpy-generated random numbers
np.random.seed(42)

# Setting the seed for Python's built-in random module
random.seed(42)

# Setting the seed for TensorFlow if used
tf.random.set_seed(42)

# Three logistic regression models are initialized with the following parameters:
# max_iter=1000: Allows the model to run for up to 1000 iterations to ensure convergence.
# solver='liblinear': Uses the 'liblinear' solver, which is efficient for small datasets.
# penalty='l1': Applies L1 regularization to encourage sparsity in the model coefficients.
# random_state=random_state: Ensures reproducibility by setting a fixed random state.
model_lr_0 = LogisticRegression(max_iter=1000,solver='liblinear',penalty='l1', random_state=random_state) 
model_lr_1 = LogisticRegression(max_iter=1000,solver='liblinear',penalty='l1', random_state=random_state) 
model_lr_2 = LogisticRegression(max_iter=1000,solver='liblinear',penalty='l1', random_state=random_state) 




# The rfe_get_score function is used to perform RFE on each model:
# For model_lr_com, 41 features are selected from the combined feature set.
# For model_lr_url, 26 features are selected from the URL feature set.
# For model_lr_html, 15 features are selected from the HTML feature set.
# The function returns the trained model, average FPR, FPR standard deviation, recall, recall standard deviation, time taken for RFE, and the RFE selector object
model_lr_com,fpr_ave_com_lr,fpr_std_com_lr,recall_com_lr,re_std_com_lr,time1_com_lr,rfe_selector_lr_com =rfe_get_score(model_lr_0,41,train_x0,train_y0,test_x0,test_y0)
model_lr_url,fpr_ave_url_lr,fpr_std_url_lr,recall_url_lr,re_std_url_lr,time1_url_lr,rfe_selector_lr_url=rfe_get_score(model_lr_1,26,train_url_x0,train_url_y0,test_url_x0,test_url_y0)
model_lr_html,fpr_ave_html_lr,fpr_std_html_lr,recall_html_lr,re_std_html_lr,time1_html_lr,rfe_selector_lr_html=rfe_get_score(model_lr_2,15,train_html_x0,train_html_y0,test_html_x0,test_html_y0)




# The get_sub_base_rf_recall function calculates the recall for each model using the selected features.
# It prints the recall values (TPR) for the combined (Fc), URL (Fu), and HTML (Fr) feature sets.
print("TPR Values")
print('Fc:')
lr_com_recall_whole=get_sub_base_rf_recall(model_lr_com,test_x0,test_y0,rfe_selector_lr_com)
print('Fu:')
lr_url_recall_whole=get_sub_base_rf_recall(model_lr_url,test_url_x0,test_url_y0,rfe_selector_lr_url)
print('Fr:')
lr_html_recall_whole=get_sub_base_rf_recall(model_lr_html,test_html_x0,test_html_y0,rfe_selector_lr_html)

print("\n\n")




# The get_base_rf_fpr function calculates the FPR for each model using the selected features.
# It prints the FPR values for the combined (Fc), URL (Fu), and HTML (Fr) feature sets.
print("FPR Values")
print('Fc:' )
base_lf_fpr_com=get_base_rf_fpr(model_lr_com,benign_test_x,benign_test_y,rfe_selector_lr_com)
print('Fu:')
base_lr_fpr_url=get_base_rf_fpr(model_lr_url,benign_test_url_x,benign_test_y,rfe_selector_lr_url)
print('Fr:')
base_lr_fpr_html=get_base_rf_fpr(model_lr_html,benign_test_html_x,benign_test_y,rfe_selector_lr_html)

TPR Values
Fc:
recall 0.84
Fu:
recall 0.26
Fr:
recall 0.77



FPR Values
Fc:
fpr 0.010
Fu:
fpr 0.010
Fr:
fpr 0.020


# No attack for WA

In [6]:
# It calls the get_sub_base_rf_recall function to calculate the recall (true positive rate) for the combined feature set model (model_lr_com) 
# on the phishing sub-test dataset (phish_sub_test_x and phish_sub_test_y).
# The rfe_selector_lr_com is used to ensure the recall is calculated using the selected features from RFE.
print('lr_no-atk_recall_Fc:')
base_sub_lr_recall_com=get_sub_base_rf_recall(model_lr_com,phish_sub_test_x,phish_sub_test_y,rfe_selector_lr_com)



# It calls the get_sub_base_rf_recall function to calculate the recall for the URL feature set model (model_lr_url) 
# on the phishing sub-test dataset (phish_sub_test_url_x and phish_sub_test_y).
# The rfe_selector_lr_url is used to ensure the recall is calculated using the selected features from RFE.
print('lr_no-atk_recall_Fu:')
base_sub_lr_recall_url=get_sub_base_rf_recall(model_lr_url,phish_sub_test_url_x,phish_sub_test_y,rfe_selector_lr_url)




# It calls the get_sub_base_rf_recall function to calculate the recall for the HTML feature set model (model_lr_html) 
# on the phishing sub-test dataset (phish_sub_test_html_x and phish_sub_test_y).
# The rfe_selector_lr_html is used to ensure the recall is calculated using the selected features from RFE.
print('lr_no-atk_recall_Fr:')
base_sub_lr_recall_html=get_sub_base_rf_recall(model_lr_html,phish_sub_test_html_x,phish_sub_test_y,rfe_selector_lr_html)
 

lr_no-atk_recall_Fc:
recall 0.87
lr_no-atk_recall_Fu:
recall 0.32
lr_no-atk_recall_Fr:
recall 0.79


# wa_u attack

In [32]:
# data_load: It loads data from a JSON file and returns it as a pandas DataFrame.
# ad_url_problem_data: This variable stores the DataFrame loaded from the specified JSON file. 
# The file path contains a subset of phishing data specifically related to URLs.
ad_url_problem_data=data_load("../data_folder/deltaphish/preprocessed/wa/wa_u/phish_sub_problem_space_url_shorten100.json")


# ad_url_problem_test_x: It extracts all columns from index 0 to 57 (58 columns in total), representing the features of the dataset.
# ad_url_problem_test_y: It extracts the column at index 58, representing the labels (e.g., phishing or benign).
ad_url_problem_test_x=ad_url_problem_data.iloc[:,0:58]
ad_url_problem_test_y=ad_url_problem_data.iloc[:,58]


#important to normalization 
# ad_url_problem_test_url_x: It concatenates columns 0 to 26 and columns 49 to 57 from the ad_url_problem_data DataFrame.
# ad_url_problem_test_html_x: This variable stores the HTML features, extracted from columns 27 to 48 of the DataFrame. 
ad_url_problem_test_url_x=pd.concat([ad_url_problem_data.iloc[:,0:27],ad_url_problem_data.iloc[:,49:58]],axis=1)
ad_url_problem_test_html_x=ad_url_problem_data.iloc[:,27:49]


# lr_recall_combined_wa_u: This calculates the recall for the logistic regression model (model_lr_com) using the combined feature set (ad_url_problem_test_x). 
# The rfe_selector_lr_com is used to ensure the model uses the selected features.

# lr_recall_url_wa_u: This calculates the recall for the logistic regression model (model_lr_url) using the URL feature set (ad_url_problem_test_url_x). 
# The rfe_selector_lr_url is used to ensure the model uses the selected features.

# lr_recall_html_wa_u: This calculates the recall for the logistic regression model (model_lr_html) using the HTML feature set (ad_url_problem_test_html_x). 
# The rfe_selector_lr_html is used to ensure the model uses the selected features.
print('lr_recall_combined_wa_Fu:')
base_sub_lr_recall_com=get_sub_base_rf_recall(model_lr_com,ad_url_problem_test_x,ad_url_problem_test_y,rfe_selector_lr_com)
print('lr_recall_url_wa_Fu:')
base_sub_lr_recall_url=get_sub_base_rf_recall(model_lr_url,ad_url_problem_test_url_x,ad_url_problem_test_y,rfe_selector_lr_url)
print('lr_recall_html_wa_Fu:')
base_sub_lr_recall_html=get_sub_base_rf_recall(model_lr_html,ad_url_problem_test_html_x,ad_url_problem_test_y,rfe_selector_lr_html)



lr_recall_combined_wa_Fu:
recall 0.58
lr_recall_url_wa_Fu:
recall 0.00
lr_recall_html_wa_Fu:
recall 0.79


# wa_r attack

In [28]:
# data_load: This function loads data from a specified JSON file and returns it as a pandas DataFrame.
# ad_problem_data: Stores the DataFrame loaded from the specified JSON file.
ad_problem_data=data_load("../data_folder/deltaphish/preprocessed/wa/wa_r/phish_sub_problem_space_html_add50internallinks.json")#wa_u,wa_c


# ad_problem_test_x: Stores the feature matrix extracted from the DataFrame. 
# It includes all columns from index 0 to 57 (58 columns in total), representing the features of the dataset.
# ad_problem_test_y: Stores the target labels extracted from the DataFrame. It includes the column at index 58, representing the labels (e.g., phishing or benign).
ad_problem_test_x=ad_problem_data.iloc[:,0:58]
ad_problem_test_y=ad_problem_data.iloc[:,58]


# ad_problem_test_url_x: Stores the URL features by concatenating columns 0 to 26 and columns 49 to 57 from the ad_problem_data DataFrame.
# ad_problem_test_html_x: Stores the HTML features extracted from columns 27 to 48 of the DataFrame.
ad_problem_test_url_x=pd.concat([ad_problem_data.iloc[:,0:27],ad_problem_data.iloc[:,49:58]],axis=1)
ad_problem_test_html_x=ad_problem_data.iloc[:,27:49]




# Recall Calculation: The function get_sub_base_rf_recall calculates the recall for each logistic regression model using the different feature sets (combined, URL, and HTML).
# Combined Features: The recall is calculated for the model model_lr_com using the combined feature set (ad_problem_test_x).
# URL Features: The recall is calculated for the model model_lr_url using the URL feature set (ad_problem_test_url_x).
# HTML Features: The recall is calculated for the model model_lr_html using the HTML feature set (ad_problem_test_html_x).
print('lr_wa_r_recall_combined:')
base_sub_lr_recall_com=get_sub_base_rf_recall(model_lr_com,ad_problem_test_x,ad_problem_test_y,rfe_selector_lr_com)
print('lr_wa_r_recall_url:')
base_sub_lr_recall_url=get_sub_base_rf_recall(model_lr_url,ad_problem_test_url_x,ad_problem_test_y,rfe_selector_lr_url)
print('lr_wa_r_recall_html:')
base_sub_lr_recall_html=get_sub_base_rf_recall(model_lr_html,ad_problem_test_html_x,ad_problem_test_y,rfe_selector_lr_html)


lr_wa_r_recall_combined:
recall 0.78
lr_wa_r_recall_url:
recall 0.32
lr_wa_r_recall_html:
recall 0.60


# wa_c attack

In [12]:
# data_load: This function is assumed to be defined earlier and it loads data from a specified JSON file into a pandas DataFrame.
ad_problem_data=data_load("../data_folder/deltaphish/preprocessed/wa/wa_c/phish_sub_problem_space_html_add50internallinks_combined_shorten_url.json")


# ad_problem_test_x: Stores the feature matrix extracted from the DataFrame, which includes columns 0 to 57 (58 columns in total).
# ad_problem_test_y: Stores the target labels extracted from the DataFrame, which is the column at index 58.
ad_problem_test_x=ad_problem_data.iloc[:,0:58]
ad_problem_test_y=ad_problem_data.iloc[:,58]
 

# ad_problem_test_url_x: Stores the URL features by concatenating columns 0 to 26 and columns 49 to 57 from the ad_problem_data DataFrame.
# ad_problem_test_html_x: Stores the HTML features extracted from columns 27 to 48 of the DataFrame.
ad_problem_test_url_x=pd.concat([ad_problem_data.iloc[:,0:27],ad_problem_data.iloc[:,49:58]],axis=1)
ad_problem_test_html_x=ad_problem_data.iloc[:,27:49]


print('lr_wa_c_recall_combined:')
base_sub_lr_recall_com=get_sub_base_rf_recall(model_lr_com,ad_problem_test_x,ad_problem_test_y,rfe_selector_lr_com)
print('lr_wa_c_recall-url:')
base_sub_lr_recall_url=get_sub_base_rf_recall(model_lr_url,ad_problem_test_url_x,ad_problem_test_y,rfe_selector_lr_url)
print('lr_wa_c_recall-html:')
base_sub_lr_recall_html=get_sub_base_rf_recall(model_lr_html,ad_problem_test_html_x,ad_problem_test_y,rfe_selector_lr_html)



lr_wa_c_recall_combined:
recall 0.44
lr_wa_c_recall-url:
recall 0.00
lr_wa_c_recall-html:
recall 0.59


# wa+_u attack

In [13]:
# data_load: This function loads data from a specified JSON file into a pandas DataFrame.
ad_problem_data=data_load("../data_folder/deltaphish/preprocessed/wa+/wa+_u/phish_sub_html_change_full_knowledge_change_again_later_only_url_shorten.json")


# ad_problem_test_x: Stores the feature matrix extracted from the DataFrame, which includes columns 0 to 57 (58 columns in total).
# ad_problem_test_y: Stores the target labels extracted from the DataFrame, which is the column at index 58.
ad_problem_test_x=ad_problem_data.iloc[:,0:58]
ad_problem_test_y=ad_problem_data.iloc[:,58]


# ad_problem_test_url_x: Stores the URL features by concatenating columns 0 to 26 and columns 49 to 57 from the ad_problem_data DataFrame.
# ad_problem_test_html_x: Stores the HTML features extracted from columns 27 to 48 of the DataFrame.
ad_problem_test_url_x=pd.concat([ad_problem_data.iloc[:,0:27],ad_problem_data.iloc[:,49:58]],axis=1)
ad_problem_test_html_x=ad_problem_data.iloc[:,27:49]

print('lr_wa+_u_recall_combined:')
base_sub_lr_recall_com=get_sub_base_rf_recall(model_lr_com,ad_problem_test_x,ad_problem_test_y,rfe_selector_lr_com)
print('lr_wa+_u_recall-url:')
base_sub_lr_recall_url=get_sub_base_rf_recall(model_lr_url,ad_problem_test_url_x,ad_problem_test_y,rfe_selector_lr_url)
print('lr_wa+_u_recall-html:')
base_sub_lr_recall_html=get_sub_base_rf_recall(model_lr_html,ad_problem_test_html_x,ad_problem_test_y,rfe_selector_lr_html)


lr_wa+_u_recall_combined:
recall 0.82
lr_wa+_u_recall-url:
recall 0.00
lr_wa+_u_recall-html:
recall 0.79


# wa+_r attack

In [15]:

ad_problem_data=data_load("../data_folder/deltaphish/preprocessed/wa+/wa+_r/phish_sub_problem_space__full_onlyhtml_316.json")#wa+_u,wa+_c

ad_problem_test_x=ad_problem_data.iloc[:,0:58]
ad_problem_test_y=ad_problem_data.iloc[:,58]

ad_problem_test_url_x=pd.concat([ad_problem_data.iloc[:,0:27],ad_problem_data.iloc[:,49:58]],axis=1)
ad_problem_test_html_x=ad_problem_data.iloc[:,27:49]
 
print('lr_sub_baseline_recall_combined:')
base_sub_lr_recall_com=get_sub_base_rf_recall(model_lr_com,ad_problem_test_x,ad_problem_test_y,rfe_selector_lr_com)
print('lr_sub_baseline_recall-url:')
base_sub_lr_recall_url=get_sub_base_rf_recall(model_lr_url,ad_problem_test_url_x,ad_problem_test_y,rfe_selector_lr_url)
print('lr_sub_baseline_recall-html:')
base_sub_lr_recall_html=get_sub_base_rf_recall(model_lr_html,ad_problem_test_html_x,ad_problem_test_y,rfe_selector_lr_html)


lr_sub_baseline_recall_combined:
recall 0.72
lr_sub_baseline_recall-url:
recall 0.32
lr_sub_baseline_recall-html:
recall 0.62


# wa+_c attack

In [16]:
ad_problem_data=data_load("../data_folder/deltaphish/preprocessed/wa+/wa+_c/phish_sub_problem_space_mix_url_html_316.json")

ad_problem_test_x=ad_problem_data.iloc[:,0:58]
ad_problem_test_y=ad_problem_data.iloc[:,58]

ad_problem_test_url_x=pd.concat([ad_problem_data.iloc[:,0:27],ad_problem_data.iloc[:,49:58]],axis=1)
ad_problem_test_html_x=ad_problem_data.iloc[:,27:49]

print('lr_wa+_c_recall_combined:')
base_sub_lr_recall_com=get_sub_base_rf_recall(model_lr_com,ad_problem_test_x,ad_problem_test_y,rfe_selector_lr_com)
print('lr_wa+_c_recall-url:')
base_sub_lr_recall_url=get_sub_base_rf_recall(model_lr_url,ad_problem_test_url_x,ad_problem_test_y,rfe_selector_lr_url)
print('lr_wa+_c_recall-html:')
base_sub_lr_recall_html=get_sub_base_rf_recall(model_lr_html,ad_problem_test_html_x,ad_problem_test_y,rfe_selector_lr_html)



lr_wa+_c_recall_combined:
recall 0.74
lr_wa+_c_recall-url:
recall 0.00
lr_wa+_c_recall-html:
recall 0.66


# pa_u attack

In [17]:

phish_sub_test_x_ch=joblib.load("../data_folder/deltaphish/preprocessed/phish_sub_test_x_ch_pa_u.pkl")

phish_sub_test_url_x_ch=pd.concat([phish_sub_test_x_ch.iloc[:,0:27],phish_sub_test_x_ch.iloc[:,49:58]],axis=1)
phish_sub_test_html_x_ch=phish_sub_test_x_ch.iloc[:,27:49]


print('lr_pa_u_recall_com:')
base_sub_lr_recall_com=get_sub_base_rf_recall(model_lr_com,phish_sub_test_x_ch,phish_sub_test_y,rfe_selector_lr_com)
print('lr_pa_u_recall_url:')
base_sub_lr_recall_url=get_sub_base_rf_recall(model_lr_url,phish_sub_test_url_x_ch,phish_sub_test_y,rfe_selector_lr_url)
print('lr_pa_u_recall_html:')
base_sub_lr_recall_html=get_sub_base_rf_recall(model_lr_html,phish_sub_test_html_x_ch,phish_sub_test_y,rfe_selector_lr_html)



lr_pa_u_recall_com:
recall 0.95
lr_pa_u_recall_url:
recall 0.94
lr_pa_u_recall_html:
recall 0.81


# pa_r attack

In [19]:
phish_sub_test_x_ch=joblib.load("../data_folder/deltaphish/preprocessed/phish_sub_test_x_ch_pa_r.pkl")

phish_sub_test_url_x_ch=pd.concat([phish_sub_test_x_ch.iloc[:,0:27],phish_sub_test_x_ch.iloc[:,49:58]],axis=1)
phish_sub_test_html_x_ch=phish_sub_test_x_ch.iloc[:,27:49]


print('lr_pa_r_recall_com:')
base_sub_lr_recall_com=get_sub_base_rf_recall(model_lr_com,phish_sub_test_x_ch,phish_sub_test_y,rfe_selector_lr_com)
print('lr_pa_r_recall_url:')
base_sub_lr_recall_url=get_sub_base_rf_recall(model_lr_url,phish_sub_test_url_x_ch,phish_sub_test_y,rfe_selector_lr_url)
print('lr_pa_r_recall_html:')
base_sub_lr_recall_html=get_sub_base_rf_recall(model_lr_html,phish_sub_test_html_x_ch,phish_sub_test_y,rfe_selector_lr_html)


lr_pa_r_recall_com:
recall 0.24
lr_pa_r_recall_url:
recall 0.32
lr_pa_r_recall_html:
recall 0.04


# pa_c attack

In [21]:
phish_sub_test_x_ch=joblib.load("../data_folder/deltaphish/preprocessed/phish_sub_test_x_ch_pa_c.pkl")

phish_sub_test_url_x_ch=pd.concat([phish_sub_test_x_ch.iloc[:,0:27],phish_sub_test_x_ch.iloc[:,49:58]],axis=1)
phish_sub_test_html_x_ch=phish_sub_test_x_ch.iloc[:,27:49]


print('lr_pa_c_recall_com:')
base_sub_lr_recall_com=get_sub_base_rf_recall(model_lr_com,phish_sub_test_x_ch,phish_sub_test_y,rfe_selector_lr_com)
print('lr_pa_c_recall_url:')
base_sub_lr_recall_url=get_sub_base_rf_recall(model_lr_url,phish_sub_test_url_x_ch,phish_sub_test_y,rfe_selector_lr_url)
print('lr_pa_c_recall_html:')
base_sub_lr_recall_html=get_sub_base_rf_recall(model_lr_html,phish_sub_test_html_x_ch,phish_sub_test_y,rfe_selector_lr_html)


lr_pa_c_recall_com:
recall 0.76
lr_pa_c_recall_url:
recall 0.94
lr_pa_c_recall_html:
recall 0.44


# ma_u attack

In [23]:
# phish_sub_test_x_ch: A copy of the original phish_sub_test_x DataFrame, which is then modified to include specific values for various URL features.
# Various columns related to URL features are set to specific values
phish_sub_test_x_ch=phish_sub_test_x.copy()
phish_sub_test_x_ch['URL_length']=-1
phish_sub_test_x_ch['URL_IP']=-1
phish_sub_test_x_ch['URL_redirect']=-1
phish_sub_test_x_ch['URL_shortener']=1
phish_sub_test_x_ch['URL_subdomains']=-1
phish_sub_test_x_ch['URL_at']=-1 
phish_sub_test_x_ch['URL_dash']=-1  
phish_sub_test_x_ch['URL_numberofCommonTerms']=-1 
phish_sub_test_x_ch['URL_checkNumerical']=-1 
phish_sub_test_x_ch['URL_checkPathExtend']=-1
phish_sub_test_x_ch['URL_checkPunycode']=-1
phish_sub_test_x_ch['URL_checkSensitiveWord']=0
phish_sub_test_x_ch['URL_checkTLDinSub']=-1
phish_sub_test_x_ch['URL_checkTLDinPath']=-1
phish_sub_test_x_ch['URL_totalWordUrl']=0
 
phish_sub_test_x_ch['URL_longestWordUrl']=0
phish_sub_test_x_ch['URL_longestWordHost']=0
phish_sub_test_x_ch['URL_longestWordPath']=0
phish_sub_test_x_ch['URL_averageWordUrl']=1000
phish_sub_test_x_ch['URL_averageWordHost']=0
phish_sub_test_x_ch['URL_averageWordPath']=1000
phish_sub_test_x_ch['REP_checkGI']=-1



# phish_sub_test_url_x_ch: Stores the URL features by concatenating columns 0 to 26 and columns 49 to 57 from the phish_sub_test_x_ch DataFrame.
# phish_sub_test_html_x_ch: Stores the HTML features extracted from columns 27 to 48 of the DataFrame.
phish_sub_test_url_x_ch=pd.concat([phish_sub_test_x_ch.iloc[:,0:27],phish_sub_test_x_ch.iloc[:,49:58]],axis=1)
phish_sub_test_html_x_ch=phish_sub_test_x_ch.iloc[:,27:49]

print('lr_ma_u_recall_com:')
base_sub_lr_recall_com=get_sub_base_rf_recall(model_lr_com,phish_sub_test_x_ch,phish_sub_test_y,rfe_selector_lr_com)
print('lr_ma_u_recall_url:')
base_sub_lr_recall_url=get_sub_base_rf_recall(model_lr_url,phish_sub_test_url_x_ch,phish_sub_test_y,rfe_selector_lr_url)
print('lr_ma_u_recall_html:')
base_sub_lr_recall_html=get_sub_base_rf_recall(model_lr_html,phish_sub_test_html_x_ch,phish_sub_test_y,rfe_selector_lr_html)



lr_ma_u_recall_com:
recall 0.00
lr_ma_u_recall_url:
recall 0.00
lr_ma_u_recall_html:
recall 0.79


# ma_r attack

In [25]:
# phish_sub_test_x_ch: A copy of the original phish_sub_test_x DataFrame, which is then modified to include specific values for various HTML features.
# Various columns related to HTML features are set to specific values
phish_sub_test_x_ch=phish_sub_test_x.copy()

phish_sub_test_x_ch['HTML_Objects']=-1
phish_sub_test_x_ch['HTML_metaScripts']=-1
phish_sub_test_x_ch['HTML_FrequentDomain']=-1
phish_sub_test_x_ch['HTML_Commonpage']=1#0#1
phish_sub_test_x_ch['HTML_CommonPageRatioinFooter']=1#1
phish_sub_test_x_ch['HTML_popUp']=-1
phish_sub_test_x_ch['HTML_RightClick']=-1
phish_sub_test_x_ch['HTML_DomainwithCopyright']=-1
phish_sub_test_x_ch['HTML_HiddenInfo_div']=-1
phish_sub_test_x_ch['HTML_nullLinksinWeb']=1
phish_sub_test_x_ch['HTML_nullLinksinFooter']=0#1
phish_sub_test_x_ch['HTML_BrokenLink']=1
phish_sub_test_x_ch['HTML_LoginForm']=-1
phish_sub_test_x_ch['HTML_HiddenInfo_input']=-1
phish_sub_test_x_ch['HTML_TitleUrlBrand']=-1
phish_sub_test_x_ch['HTML_favicon']=-1
phish_sub_test_x_ch['HTML_statusBarMod']=-1
phish_sub_test_x_ch['HTML_css']=-1
phish_sub_test_x_ch['HTML_anchors']=-1



# phish_sub_test_url_x_ch: Stores the URL features by concatenating columns 0 to 26 and columns 49 to 57 from the phish_sub_test_x_ch DataFrame.
# phish_sub_test_html_x_ch: Stores the HTML features extracted from columns 27 to 48 of the DataFrame.
phish_sub_test_url_x_ch=pd.concat([phish_sub_test_x_ch.iloc[:,0:27],phish_sub_test_x_ch.iloc[:,49:58]],axis=1)
phish_sub_test_html_x_ch=phish_sub_test_x_ch.iloc[:,27:49]


print('lr_ma_r_recall_com:')
base_sub_lr_recall_com=get_sub_base_rf_recall(model_lr_com,phish_sub_test_x_ch,phish_sub_test_y,rfe_selector_lr_com)
print('lr_ma_r_recall_url:')
base_sub_lr_recall_url=get_sub_base_rf_recall(model_lr_url,phish_sub_test_url_x_ch,phish_sub_test_y,rfe_selector_lr_url)
print('lr_ma_r_recall_html:')
base_sub_lr_recall_html=get_sub_base_rf_recall(model_lr_html,phish_sub_test_html_x_ch,phish_sub_test_y,rfe_selector_lr_html)


lr_ma_r_recall_com:
recall 0.00
lr_ma_r_recall_url:
recall 0.32
lr_ma_r_recall_html:
recall 0.00


# ma_c attack

In [34]:
phish_sub_test_x_ch=phish_sub_test_x.copy()
phish_sub_test_x_ch['URL_length']=-1
phish_sub_test_x_ch['URL_IP']=-1#single no change
phish_sub_test_x_ch['URL_redirect']=-1#
phish_sub_test_x_ch['URL_shortener']=1 #single no change
phish_sub_test_x_ch['URL_subdomains']=-1
phish_sub_test_x_ch['URL_at']=-1 
phish_sub_test_x_ch['URL_dash']=-1  
phish_sub_test_x_ch['URL_numberofCommonTerms']=-1 
phish_sub_test_x_ch['URL_checkNumerical']=-1 #single no change
phish_sub_test_x_ch['URL_checkPathExtend']=-1#single no change
phish_sub_test_x_ch['URL_checkPunycode']=-1#single no change
phish_sub_test_x_ch['URL_checkSensitiveWord']=0
phish_sub_test_x_ch['URL_checkTLDinSub']=-1
phish_sub_test_x_ch['URL_checkTLDinPath']=-1
phish_sub_test_x_ch['URL_totalWordUrl']=0

phish_sub_test_x_ch['URL_longestWordUrl']=0
phish_sub_test_x_ch['URL_longestWordHost']=0 
phish_sub_test_x_ch['URL_longestWordPath']=0
phish_sub_test_x_ch['URL_averageWordUrl']=1000
phish_sub_test_x_ch['URL_averageWordHost']=0
phish_sub_test_x_ch['URL_averageWordPath']=1000
 
phish_sub_test_x_ch['HTML_Objects']=-1
phish_sub_test_x_ch['HTML_metaScripts']=-1
phish_sub_test_x_ch['HTML_FrequentDomain']=-1
phish_sub_test_x_ch['HTML_Commonpage']=1#0#1
phish_sub_test_x_ch['HTML_CommonPageRatioinFooter']=1#1
 
phish_sub_test_x_ch['HTML_popUp']=-1
phish_sub_test_x_ch['HTML_RightClick']=-1
phish_sub_test_x_ch['HTML_DomainwithCopyright']=-1
phish_sub_test_x_ch['HTML_HiddenInfo_div']=-1
phish_sub_test_x_ch['HTML_nullLinksinWeb']=1#1#0
phish_sub_test_x_ch['HTML_nullLinksinFooter']=0#1
phish_sub_test_x_ch['HTML_BrokenLink']=1
phish_sub_test_x_ch['HTML_LoginForm']=-1
phish_sub_test_x_ch['HTML_HiddenInfo_input']=-1
phish_sub_test_x_ch['HTML_TitleUrlBrand']=-1
phish_sub_test_x_ch['HTML_favicon']=-1
phish_sub_test_x_ch['HTML_statusBarMod']=-1
phish_sub_test_x_ch['HTML_css']=-1
phish_sub_test_x_ch['HTML_anchors']=-1
phish_sub_test_x_ch['REP_checkGI']=-1



# phish_sub_test_url_x_ch: Stores the URL features by concatenating columns 0 to 26 and columns 49 to 57 from the phish_sub_test_x_ch DataFrame.
# phish_sub_test_html_x_ch: Stores the HTML features extracted from columns 27 to 48 of the DataFrame.
phish_sub_test_url_x_ch=pd.concat([phish_sub_test_x_ch.iloc[:,0:27],phish_sub_test_x_ch.iloc[:,49:58]],axis=1)
phish_sub_test_html_x_ch=phish_sub_test_x_ch.iloc[:,27:49]


print('lr_ma_c_recall_com:')
base_sub_lr_recall_com=get_sub_base_rf_recall(model_lr_com,phish_sub_test_x_ch,phish_sub_test_y,rfe_selector_lr_com)
print('lr_ma_c_recall_url:')
base_sub_lr_recall_url=get_sub_base_rf_recall(model_lr_url,phish_sub_test_url_x_ch,phish_sub_test_y,rfe_selector_lr_url)
print('lr_mc_c_recall_html:')
base_sub_lr_recall_html=get_sub_base_rf_recall(model_lr_html,phish_sub_test_html_x_ch,phish_sub_test_y,rfe_selector_lr_html)



lr_ma_c_recall_com:
recall 0.00
lr_ma_c_recall_url:
recall 0.00
lr_mc_c_recall_html:
recall 0.00
